In [ ]:
import random
from pathlib import Path  # 경로 관리를 위한 모듈

# 지정된 폴더 내에서 각 클래스별 이미지를 일정 개수(keep)로 줄이는 함수
def downsample_folder(root_dir, classes, keep=5000):
    root = Path(root_dir)  # 문자열 경로를 Path 객체로 변환
    for cls in classes:
        cls_dir = root / cls  # 클래스 폴더 경로 생성
        # .jpg 또는 .png 확장자를 가진 모든 파일 목록 생성
        imgs = list(cls_dir.rglob("*.jpg || *.png"))
        
        # 이미지 개수가 keep 이하이면 변경할 필요 없음
        if len(imgs) <= keep:
            print(f"{cls}: {len(imgs)} images (no change)")
            continue

        # keep개를 제외한 나머지 이미지를 무작위로 선택하여 삭제
        to_remove = random.sample(imgs, len(imgs) - keep)
        for f in to_remove:
            f.unlink()  # 파일 삭제
        print(f"{cls}: removed {len(to_remove)} images, {keep} remain")


In [ ]:
# 데이터가 저장된 디렉토리와 클래스 리스트 설정
data_dir = './face_crops'
CLASSES = ['angry', 'happy', 'neutral', 'sad']

# downsample_folder 함수를 호출하여 각 클래스별 이미지 수를 줄임
downsample_folder(data_dir, CLASSES, keep=5000)
print("Downsampling complete!")

In [ ]:
import cv2
from mtcnn import MTCNN  # 얼굴 검출을 위한 MTCNN 모델
from pathlib import Path
import os
import tensorflow as tf  # GPU 사용 및 메모리 관리
from PIL import Image  # 이미지 파일 열기용 라이브러리
import numpy as np

# GPU 메모리 설정
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        # GPU 메모리를 필요에 따라 점진적으로 할당하도록 설정
        tf.config.experimental.set_memory_growth(gpu, True)

# 전처리 작업에 사용할 상수 설정
EYE_THRESHOLD = 0.52  # 눈 위치 비율 임계값
MAX_WIDTH = 640       # 리사이즈 시 최대 너비

# 데이터 및 출력 경로 설정
BASE_DIR = Path("./data")
OUTPUT_DIR = BASE_DIR.parent / "face_crops"

# 얼굴 클래스 리스트
CLASSES = ["angry", "happy", "neutral", "sad"]

# MTCNN 얼굴 검출기를 한 번만 생성 (여러 함수에서 공유)
detector = MTCNN()


In [ ]:
def load_and_resize(path):
    try:
        # 파일 열기 전, 파일 무결성을 확인 (pil.verify)
        with Image.open(path) as pil:
            pil.verify()
        # 파일을 다시 열고 RGB 모드로 변환
        with Image.open(path) as pil:
            pil = pil.convert("RGB")
    except Exception:
        # 파일에 문제가 있으면 삭제 처리
        try:
            os.remove(path)
            print(f"Deleted corrupt image: {path}")
        except Exception as e:
            print(f"Could not delete {path}: {e}")
        return None

    # PIL 이미지를 numpy 배열로 변환
    img = np.array(pil)
    # 색상 순서를 OpenCV 방식(BGR)로 변경
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    # 이미지의 크기 체크 및 리사이즈 (너비가 MAX_WIDTH를 초과할 경우)
    h, w = img.shape[:2]
    if w > MAX_WIDTH:
        scale = MAX_WIDTH / w
        img = cv2.resize(img, (int(w * scale), int(h * scale)))
    return img

In [ ]:
def extract_face(img):
    # 얼굴 검출 (MTCNN은 얼굴이 없으면 빈 리스트 반환)
    faces = detector.detect_faces(img)
    if not faces:
        return None
    # 첫 번째 검출된 얼굴 정보 사용
    box = faces[0]['box']
    key = faces[0]['keypoints']
    # 왼쪽 눈과 오른쪽 눈의 y좌표 평균 계산
    eye_y = (key['left_eye'][1] + key['right_eye'][1]) / 2

    # 눈 위치가 이미지 하단에 너무 가까우면 (비율이 임계값보다 크면) 이미지 상하 반전 시도
    if eye_y / img.shape[0] > EYE_THRESHOLD:
        img = cv2.flip(img, 0)  # 상하 반전
        faces = detector.detect_faces(img)
        if not faces:
            return None
        box = faces[0]['box']

    # 얼굴 영역의 박스 정보를 이용하여 정사각형 영역 계산
    x, y, w, h = box
    side = max(w, h)
    cx, cy = x + w // 2, y + h // 2
    half = side // 2

    # 얼굴 영역이 이미지 범위를 벗어나지 않도록 좌표 계산
    y1 = max(0, cy - half)
    y2 = min(img.shape[0], cy + half)
    x1 = max(0, cx - half)
    x2 = min(img.shape[1], cx + half)
    return img[y1:y2, x1:x2]

In [ ]:
# 멀티스레드 처리를 위한 ThreadPoolExecutor
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# process_file: 단일 이미지 파일 처리 함수
def process_file(args):
    cls, path, idx = args
    # 출력 폴더가 없으면 생성 (클래스별 폴더)
    dst = OUTPUT_DIR / cls
    dst.mkdir(parents=True, exist_ok=True)

    # 이미지 로딩 및 리사이즈
    img = load_and_resize(path)
    if img is None:
        return

    # 얼굴 영역 추출
    face = extract_face(img)
    if face is not None:
        # 파일 이름 형식: 클래스명_0001.jpg
        out_name = f"{cls}_{idx:04d}.jpg"
        cv2.imwrite(str(dst / out_name), face)  # 추출된 얼굴 이미지를 저장

# 전체 데이터셋에 대해 멀티스레딩으로 얼굴 추출을 실행
def run_face_extraction():
    # 전체 출력 폴더 생성
    OUTPUT_DIR.mkdir(exist_ok=True)
    # 각 클래스별로 파일 목록을 수집하여 작업 리스트 생성
    tasks = [(cls, p, i) 
             for cls in CLASSES 
             for i, p in enumerate((BASE_DIR / cls).rglob("*.*"), start=1)]
    
    # ThreadPoolExecutor를 사용하여 병렬 처리
    with ThreadPoolExecutor() as executor:
        list(tqdm(executor.map(process_file, tasks), total=len(tasks), desc="Processing all"))
    
    print("All done!")


In [ ]:
# 얼굴 추출 및 저장을 실행
run_face_extraction()

In [ ]:
import cv2
from pathlib import Path

# 이미지 파일을 읽어 흑백으로 변환하고, 지정한 대상 경로에 저장하는 함수
def convert_to_grayscale(src_img_path, dest_img_path):
    img = cv2.imread(str(src_img_path))
    if img is None:
        return False
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(str(dest_img_path), gray)
    return True

BASE = Path("./face_crops")
BASE_GRAY = Path("./face_crops_gray")
BASE_GRAY.mkdir(parents=True, exist_ok=True)

count = 0

# JPG 파일 처리
for img_path in BASE.rglob("*.jpg"):
    # 변환 후 동일한 파일 이름으로 저장
    dest_path = BASE_GRAY / img_path.name
    if convert_to_grayscale(img_path, dest_path):
        count += 1

# PNG 파일 처리
for img_path in BASE.rglob("*.png"):
    dest_path = BASE_GRAY / img_path.name
    if convert_to_grayscale(img_path, dest_path):
        count += 1

print(f"Converted {count} images to grayscale, saved in {BASE_GRAY}")


In [ ]:
import pandas as pd

# CSV 파일을 읽어 문자열 라벨을 정수로 매핑하는 함수
def map_labels(csv_path, label_map):
    df = pd.read_csv(csv_path)
    # "label" 컬럼의 문자열을 label_map에 따라 정수로 변환
    df["label"] = df["label"].map(label_map)
    return df

# DataFrame을 CSV 파일로 저장하는 함수
def save_csv(df, csv_path):
    df.to_csv(csv_path, index=False)


In [ ]:
# 처리할 CSV 파일 경로
csv_path = "crops_dataset.csv"
# 문자열 라벨과 정수 값 매핑
label_map = {"angry": 0, "happy": 1, "neutral": 2, "sad": 3}

# CSV 파일의 라벨을 매핑
df = map_labels(csv_path, label_map)
# 매핑된 라벨 값들의 분포를 출력
print(df["label"].value_counts())
# 변경된 DataFrame을 다시 CSV 파일로 저장
save_csv(df, csv_path)
print("CSV label mapping complete.")
